# Install

In [ ]:
!pip install einops datasets jaxtyping better_abc fancy_einsum wandb netcal

# Setup

In [ ]:
import sys
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
path_to_root = '/content/drive/My Drive/Colab Notebooks/BatuEl_Dissertation'
sys.path.append(path_to_root)
print("Drive mounted.")

data_path = path_to_root + '/data'

In [ ]:
import torch
import tqdm
from reprshift.learning.algorithms import ERM
from reprshift.models.hparams import hparams_f
from reprshift.dataset.datasets import MultiNLI, CivilComments
from reprshift.dataset.dataloaders import InfiniteDataLoader, FastDataLoader

from reprshift.models.model_param_maps import ERM_to_HookedEncoder, load_focal, load_groupdro, load_jtt, load_lff
from reprshift.models.HookedEncoderConfig import bert_config

from transformer_lens2 import HookedEncoder, HookedTransformerConfig
import numpy as np
import pandas as pd

# Dataset

In [ ]:
DATASET = 'MultiNLI'  # 'CivilComments' , 'MultiNLI'
SEED = 1

if DATASET == 'MultiNLI':
    NUM_CLASSES = 3
    NUM_ATTRIBUTES = 2
    # train_dataset = MultiNLI(data_path, 'tr',  hparams=hparams_f('ERM'))
    val_dataset = MultiNLI(data_path, 'va', hparams=hparams_f('ERM'))
    te_dataset = MultiNLI(data_path, 'te', hparams=hparams_f('ERM'))
    models_path = path_to_root + '/models/models_mnli'
    representations_path = path_to_root + '/representations/representations_mnli'
    print(DATASET)
elif DATASET  == 'CivilComments':
    NUM_CLASSES = 2
    NUM_ATTRIBUTES = 8
    # train_dataset = CivilComments(data_path, 'tr',  hparams=hparams_f('ERM'), granularity="fine")
    val_dataset = CivilComments(data_path, 'va', hparams=hparams_f('ERM'))
    # te_dataset = CivilComments(data_path, 'te', hparams=hparams_f('ERM'))
    models_path = path_to_root + '/models/models_civilcomments'
    representations_path = path_to_root + '/representations/representations_civilcomments'
    print(DATASET)
else:
    print('Dataset Not Implemented')

# Models

In [ ]:
valmetrics = torch.load(path_to_root + f'/results/ValidationMetrics/clean_val_results.pth')
CURRENT_BEST_EPOCH = valmetrics[DATASET][SEED]['A Selection']
### MODELS ###
MODELS  = {'pretrained': {'path': models_path + '/00_randominit/',  'load_f': lambda x: x, 'epoch':0,},
          'erm': {'path': models_path + '/01_erm/',  'load_f': lambda x: x, 'epoch':CURRENT_BEST_EPOCH['erm'],},
          'groupdro': {'path': models_path + '/03_groupdro/',  'load_f': load_groupdro, 'epoch':CURRENT_BEST_EPOCH['groupdro'],},
          'jtt': {'path': models_path + '/06_jtt/',  'load_f': load_jtt, 'epoch':CURRENT_BEST_EPOCH['jtt'],},
          'lff': {'path': models_path + '/07_lff/',  'load_f': load_lff, 'epoch':CURRENT_BEST_EPOCH['lff'],},
          'focal': {'path': models_path + '/15_focal/',  'load_f': lambda x: x, 'epoch':CURRENT_BEST_EPOCH['focal'],},}

def load_and_init(algorithm_name, seed):
    state_dict_PATH = MODELS[algorithm_name]['path']
    load_f =  MODELS[algorithm_name]['load_f']
    epoch = MODELS[algorithm_name]['epoch']
    algorithm_state_dict_PATH = state_dict_PATH + f'seed{seed}/sd_epoch{epoch}.pth'
    sd = load_f(torch.load(algorithm_state_dict_PATH))
    ### Initialize ERM Model ###
    # hparams = hparams_f('ERM')
    # algorithm = ERM(num_classes=NUM_CLASSES, num_attributes=NUM_ATTRIBUTES, hparams=hparams)
    # algorithm.load_state_dict(sd)
    ### Load ERM Model ###
    bert = HookedEncoder(HookedTransformerConfig(**bert_config(NUM_CLASSES)))
    bert.load_state_dict(ERM_to_HookedEncoder(sd, bert.state_dict()))
    return bert

In [ ]:
bert = load_and_init('erm', SEED).cuda().eval()
# Set requires_grad to False for all parameters of BERT
for param in bert.parameters():
    param.requires_grad = False

In [ ]:
CURRENT_BEST_EPOCH

# Cache Representations

In [ ]:
if DATASET == 'CivilComments':
    per_group_repr = 150
if DATASET == 'MultiNLI':
    per_group_repr = 600

unique_y = [i for i in range(NUM_CLASSES)]
unique_a = [i for i in range(NUM_ATTRIBUTES)]
y_s = [f'y{i}' for i in range(NUM_CLASSES)]
a_s = [f'a{i}' for i in range(NUM_ATTRIBUTES)]

REPR_KEYS = ['blocks.11.ln1.hook_normalized', 'blocks.11.mlp.hook_post' , 'blocks.11.hook_mlp_out', 'blocks.11.ln2.hook_normalized']
REPRS = {repr_key: {f'y{y_idx}':{f'a{a_idx}':[] for a_idx in unique_a}  for y_idx in unique_y} for repr_key in REPR_KEYS}
LOGITS = {f'y{y_idx}':{f'a{a_idx}':[] for a_idx in unique_a} for y_idx in unique_y}
LOGITS, REPRS

In [ ]:
SPLIT = 'va'  # 'tr', 'va', 'te'

# if SPLIT == 'tr':
#     dataset = train_dataset
# elif SPLIT == 'va':
#     dataset = val_dataset
# elif SPLIT == 'te':
#     dataset = te_dataset
# else:
#     print('Split Unavailable')

# for Y_CURR in unique_y:
#     for A_CURR in unique_a:
#         print(f'y{Y_CURR}', f'a{A_CURR}')
#         val_loader = FastDataLoader(  dataset=dataset,
#                               batch_size=32,
#                               num_workers=1,
#                               )
#         train_minibatches_iterator = iter(val_loader)

#         for step in tqdm.tqdm(range(len(val_loader))):
#             total_group_members = np.sum([batch.shape[0] for batch in LOGITS[f'y{Y_CURR}'][f'a{A_CURR}']])
#             if total_group_members > per_group_repr:
#                 break
#             i, x, y, a = next(train_minibatches_iterator)

#             A_MASK = (a == A_CURR)
#             Y_MASK = (y == Y_CURR)
#             MASK = A_MASK & Y_MASK
#             x = x[MASK]
#             input_ids = x[:,:,0].cuda()
#             one_zero_attention_mask = x[:,:,1].cuda()
#             token_type_ids = x[:,:,2].cuda()

#             with torch.no_grad():
#                 logits, cache = bert.run_with_cache(input_ids, one_zero_attention_mask=one_zero_attention_mask, token_type_ids=token_type_ids)
#                 LOGITS[f'y{Y_CURR}'][f'a{A_CURR}'].append(logits[:,0,:].detach().cpu())
#                 for repr_key in REPR_KEYS:
#                     REPRS[repr_key][f'y{Y_CURR}'][f'a{A_CURR}'].append(cache[repr_key][:,0,:].detach().cpu())
#             del logits, cache
#             torch.cuda.empty_cache()
#         LOGITS[f'y{Y_CURR}'][f'a{A_CURR}'] = torch.cat(LOGITS[f'y{Y_CURR}'][f'a{A_CURR}'])[:per_group_repr]
#         for repr_key in REPR_KEYS:
#             REPRS[repr_key][f'y{Y_CURR}'][f'a{A_CURR}'] =  torch.cat(REPRS[repr_key][f'y{Y_CURR}'][f'a{A_CURR}'])[:per_group_repr]

# torch.save(LOGITS, representations_path + f'/ModelEdit/LOGITS_{SPLIT}_seed{SEED}' )
# torch.save(REPRS, representations_path + f'/ModelEdit/REPRS_{SPLIT}_seed{SEED}' )

In [ ]:
LOGITS = torch.load(representations_path + f'/ModelEdit/LOGITS_{SPLIT}_seed{SEED}')
REPRS  = torch.load(representations_path + f'/ModelEdit/REPRS_{SPLIT}_seed{SEED}' )
REPRS_TRAIN  = torch.load(representations_path + f'/ModelEdit/REPRS_tr_seed{SEED}' )

In [ ]:
### Sanity Check:  bert.W_out[11] ###
mlp_out = lambda x: (x.cuda()  @ bert.W_out[11]) +  bert.b_out[11]
mlp_out(REPRS['blocks.11.mlp.hook_post']['y0']['a0']).shape, REPRS['blocks.11.hook_mlp_out']['y0']['a0'].shape
REPRS['blocks.11.hook_mlp_out']['y0']['a0'].cuda().round(decimals=4) == mlp_out(REPRS['blocks.11.mlp.hook_post']['y0']['a0']).round(decimals=4)

In [ ]:
### Sanity Check:  Logit Lens ###
def Layer11Norm2(res):
    res = bert.blocks[11].ln2(res.unsqueeze(dim=1).cuda())
    return res
def LogitLens(res):
    res = bert.mlm_head(res)
    res = bert.unembed(res)
    return res
def LogitLenswLayer11Norm2(res):
    res = bert.blocks[11].ln2(res.unsqueeze(dim=1).cuda())
    res = bert.mlm_head(res)
    res = bert.unembed(res)
    return res[:,0,:]

# Implementation with Matrix Multiplication
def logit_lens_with_norm(cls_reprs, sd=bert.state_dict()):
  cls_reprs = cls_reprs.cuda()
  mean = cls_reprs.mean(dim=1, keepdim=True)
  std = cls_reprs.std(dim=1, keepdim=True)
  cls_reprs = (cls_reprs - mean) / (std)
  cls_reprs = (sd['blocks.11.ln2.w'] * cls_reprs) + sd['blocks.11.ln2.b']
  out = cls_reprs @ sd['mlm_head.W'].T
  out = out + sd['mlm_head.b']
  out = torch.tanh(out)
  out = out @ sd['unembed.W_U']
  out = out + sd['unembed.b_U']
  return out

resid_post = REPRS['blocks.11.ln1.hook_normalized']['y0']['a0'] + REPRS['blocks.11.hook_mlp_out']['y0']['a0']
# Implementation with Matrix Multiplication
# logit_lens_with_norm(resid_post)[:,:], LOGITS['y0']['a0']
# Implementation with Components
LogitLenswLayer11Norm2(resid_post), LOGITS['y0']['a0']

# Step 1: Calculating the Key

The key is the representation of the shortcut in the internal representation of the model. In the ROME paper, this is calculated by passing a set of sequences that contain the key through the model and caching the representations. Then, the representations are averaged to calculate the key $k^{*}$.

Here, we are aiming to capture the representation of the shortcut. Hence, we pass the data with and without the shortcut through the model and cache the activations. As a key, we use two alternatives:

1. The average of the representations from examples with shortcuts.
2. The difference between the average of examples with and without shortcut.

In [ ]:
### Define Keys ###
a0_hook_post = torch.cat([REPRS['blocks.11.mlp.hook_post'][f'y{label_idx}']['a0'] for label_idx in [0,1,2]])
a1_hook_post = torch.cat([REPRS['blocks.11.mlp.hook_post'][f'y{label_idx}']['a1'] for label_idx in [0,1,2]])
key_mean = a1_hook_post.mean(dim=0)
key_diff = a1_hook_post.mean(dim=0) - a0_hook_post.mean(dim=0)
print(key_mean.shape, key_diff.shape)
### The Key for Shortcut ###
k_star = key_mean

# Step 2: Calculating the Value

This is the value that needs to be written on the residual stream in order to decrease the logit of the contradiction class. This can be computed by solving a simple optimization problem:

What is the vector that needs to be added to the residual stream so that the logits that correspond to the contradiction class are suppressed?

As an additional contraint, we add the requirement that the logits that correspond to non-shortcut examples are not changed.

In [ ]:
resid_post = REPRS['blocks.11.ln1.hook_normalized']['y0']['a0'] + REPRS['blocks.11.hook_mlp_out']['y0']['a0']
res = LogitLenswLayer11Norm2(resid_post)
res, LOGITS['y0']['a0']

In [ ]:
# import torch
# import torch.nn as nn

# def LogitLenswLayer11Norm2(res):
#     res = bert.blocks[11].ln2(res.unsqueeze(dim=1).cuda())
#     res = bert.mlm_head(res)
#     res = bert.unembed(res)
#     return res[:,0,:]

# class Model(nn.Module):
#     def __init__(self, d_model):
#         super(Model, self).__init__()
#         self.param_vector = nn.Parameter(torch.randn(d_model).to(torch.float32))

#     def forward(self, x):
#         x = x + self.param_vector
#         x = LogitLenswLayer11Norm2(x)
#         return x

# model = Model(768).cuda()
# optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.1)

# # Objective 1: Minimize Contradiction Logit
# # a0_ln1_hook = torch.cat([REPRS['blocks.11.ln1.hook_normalized'][f'y{label_idx}']['a0'] for label_idx in [0,1,2]]).cuda()
# a1_ln1_hook = torch.cat([REPRS['blocks.11.ln1.hook_normalized'][f'y{label_idx}']['a1'] for label_idx in [0,1,2]]).cuda()

# # Objective 2: Keep the other Logits Constant
# a1_logits = torch.cat([LOGITS[f'y{label_idx}']['a1'] for label_idx in [0,1,2]]).cuda()

# losses = []
# # Training loop
# for _ in tqdm.tqdm(range(100)):
#     optimizer.zero_grad()
#     # Forward pass
#     loss = 0
#     output = model(a1_ln1_hook)
#     loss = output[:, 0].sum()
#     # control_loss = (output[:, :] - a1_logits[:, :]).norm()
#     # loss = loss + control_loss
#     loss.backward()
#     optimizer.step()
#     losses.append(loss)
#     # print(loss.item())

# ### v_star ###
# v_star = model.param_vector

In [ ]:
#############################
#############################
#### ALTERNATIVE 2 ##########
#############################
#############################

import torch
import torch.nn as nn

def LogitLenswLayer11Norm2(res):
    res = bert.blocks[11].ln2(res.unsqueeze(dim=1).cuda())
    res = bert.mlm_head(res)
    res = bert.unembed(res)
    return res[:,0,:]

class Model(nn.Module):
    def __init__(self, d_model):
        super(Model, self).__init__()
        self.param_vector = nn.Parameter(torch.randn(d_model).to(torch.float32))

    def forward(self, x):
        x = x + self.param_vector
        x = LogitLenswLayer11Norm2(x)
        return x

model = Model(768).cuda()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.1)

# Objective 1: Minimize Contradiction Logit
# a0_ln1_hook = torch.cat([REPRS['blocks.11.ln1.hook_normalized'][f'y{label_idx}']['a0'] for label_idx in [0,1,2]]).cuda()
a1_ln1_hook = torch.cat([REPRS['blocks.11.ln1.hook_normalized'][f'y{label_idx}']['a1'] for label_idx in [0,1,2]]).cuda()
a0_ln1_hook = torch.cat([REPRS['blocks.11.ln1.hook_normalized'][f'y{label_idx}']['a0'] for label_idx in [0,1,2]]).cuda()

# Objective 2: Keep the other Logits Constant
a1_logits = torch.cat([LOGITS[f'y{label_idx}']['a1'] for label_idx in [0,1,2]]).cuda()
a0_logits = torch.cat([LOGITS[f'y{label_idx}']['a1'] for label_idx in [0,1,2]]).cuda()

losses = []

LogitShiftFactor = {0:4.1, 1:4.5, 2:4.1}


for _ in tqdm.tqdm(range(1000)):
    optimizer.zero_grad()
    loss = 0
    output = model(a1_ln1_hook)
    # loss = output[:, 0].sum()
    a1_control_loss = (output[:, :] - (a1_logits - torch.tensor([LogitShiftFactor[SEED], 0, 0]).cuda())).norm()

    output_a0 = model(a0_ln1_hook)
    a0_control_loss = (output_a0[:, :] - a0_logits[:, :]).norm()

    loss = a1_control_loss + a0_control_loss
    loss.backward()
    optimizer.step()
    losses.append(loss.item())

v_star = model.param_vector

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.plot(losses, marker='o', linestyle='-', color='b')

plt.title('Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.grid(True)

plt.show()


In [ ]:
alpha = 1

### What does v_star do to the logits?
for y_idx in [0,1,2]:
    print('#################')
    for a_idx in ['a0', 'a1']:
        print(f'y{y_idx}', a_idx)
        resid_post = REPRS['blocks.11.ln1.hook_normalized'][f'y{y_idx}'][a_idx] + REPRS['blocks.11.hook_mlp_out'][f'y{y_idx}'][a_idx]
        prev = LogitLenswLayer11Norm2(resid_post).detach().cpu()
        resid_post = REPRS['blocks.11.ln1.hook_normalized'][f'y{y_idx}'][a_idx] + (alpha * torch.stack([v_star for i in range(600)]).cpu())
        post = LogitLenswLayer11Norm2(resid_post).detach().cpu()
        print('prev mean logit:', prev.mean(dim=0))
        print('post mean logit:', post.mean(dim=0))

        print('prev accuracy:', (prev.argmax(dim=1) == y_idx).sum() / 600)
        print('post accuracy:', (post.argmax(dim=1) == y_idx).sum() / 600)

# Step 3: Rank One Edit

In [ ]:
### K-star: the key input ###
K_star = k_star.cuda()
### V-star: the value output ###
V_star = v_star.cuda()
### W: Pre-edit Weight Matrix ###
W = bert.W_out[11].detach().cuda().T

##############################
#### Pre-cached C #############
##############################
Kcutoff = 999999
### K Matrix: Each column is a key ###
K_a0 = torch.cat([REPRS['blocks.11.mlp.hook_post'][f'y{label_idx}']['a0'][:Kcutoff] for label_idx in [0,1,2]])
K_a1 = torch.cat([REPRS['blocks.11.mlp.hook_post'][f'y{label_idx}']['a1'][:Kcutoff] for label_idx in [0,1,2]])
K = torch.cat([K_a0, K_a1])[:, :].T.cuda()
### C Matrix: K @ K.T ###
C = K @ K.T

### Lambda:  Λ = (v∗ −Wk∗)/(C−1k∗)T k∗ ###
Lambda_numerator = (V_star - W @ K_star)
Lambda_denominator = (C.inverse() @ K_star).T @ K_star
Lambda = Lambda_numerator / Lambda_denominator
### W_hat ###
W.shape, Lambda.shape, C.shape, K_star.shape
W_hat = W + (Lambda.unsqueeze(1) @ (C.inverse() @ K_star).unsqueeze(dim=1).T)
W_edit = W_hat.T

In [ ]:
import copy
bert_updated = copy.deepcopy(bert)
sd = bert.state_dict()
sd['blocks.11.mlp.W_out'] = W_edit
bert_updated.load_state_dict(sd)

# Evaluation Val

In [ ]:
LOGITS_EDIT_VAL = {f'y{y_idx}':{f'a{a_idx}':[] for a_idx in unique_a} for y_idx in unique_y}
dataset=val_dataset

for Y_CURR in unique_y:
    for A_CURR in unique_a:
        print(f'y{Y_CURR}', f'a{A_CURR}')
        val_loader = FastDataLoader(  dataset=dataset,
                              batch_size=32,
                              num_workers=1,
                              )
        train_minibatches_iterator = iter(val_loader)

        for step in tqdm.tqdm(range(len(val_loader))):

            total_group_members = np.sum([batch.shape[0] for batch in LOGITS_EDIT_VAL[f'y{Y_CURR}'][f'a{A_CURR}']])
            if total_group_members > per_group_repr:
                break
            i, x, y, a = next(train_minibatches_iterator)

            A_MASK = (a == A_CURR)
            Y_MASK = (y == Y_CURR)
            MASK = A_MASK & Y_MASK
            x = x[MASK]
            input_ids = x[:,:,0].cuda()
            one_zero_attention_mask = x[:,:,1].cuda()
            token_type_ids = x[:,:,2].cuda()

            with torch.no_grad():
                logits = bert_updated(input_ids, one_zero_attention_mask=one_zero_attention_mask, token_type_ids=token_type_ids)
                LOGITS_EDIT_VAL[f'y{Y_CURR}'][f'a{A_CURR}'].append(logits[:,0,:].detach().cpu())
            del logits
            torch.cuda.empty_cache()

        LOGITS_EDIT_VAL[f'y{Y_CURR}'][f'a{A_CURR}'] = torch.cat(LOGITS_EDIT_VAL[f'y{Y_CURR}'][f'a{A_CURR}'])[:per_group_repr]

## Confusion Matrix

In [ ]:
# Initialize confusion matrices
conf_matrix_a0 = np.zeros((NUM_CLASSES, NUM_CLASSES), dtype=int)
conf_matrix_a1 = np.zeros((NUM_CLASSES, NUM_CLASSES), dtype=int)

conf_matrix = {ai: np.zeros((NUM_CLASSES, NUM_CLASSES), dtype=int) for ai in a_s}

# Compute confusion matrix for a0 and a1
# for ai in a_s:
conf_matrix_dict = {}

for ai in a_s:
    for true_label in range(NUM_CLASSES):
        logits_ai = LOGITS_EDIT_VAL[y_s[true_label]][ai]
        pred_labels_ai = logits_ai.argmax(dim=1).numpy()
        for pred_label in pred_labels_ai:
            conf_matrix[ai][true_label, pred_label] += 1
    conf_matrix_dict[ai] = pd.DataFrame(conf_matrix[ai], columns=[f'Pred y{i}' for i in range(NUM_CLASSES)], index=[f'True y{i}' for i in range(NUM_CLASSES)]).T / per_group_repr * 100

accuracy_df = pd.concat(conf_matrix_dict, axis=1).round(2)

### For Validation ###
accuracy_df_PATH = path_to_root + f'/results/ModelEdits/Final/ConfusionMatrixVal_{DATASET}_seed{SEED}'
accuracy_df.to_csv(accuracy_df_PATH)

In [ ]:
accuracy_df

# Evaluation Test

In [ ]:
LOGITS_EDIT_TEST = {f'y{y_idx}':{f'a{a_idx}':[] for a_idx in unique_a} for y_idx in unique_y}
dataset=te_dataset

for Y_CURR in unique_y:
    for A_CURR in unique_a:
        print(f'y{Y_CURR}', f'a{A_CURR}')
        val_loader = FastDataLoader(  dataset=dataset,
                              batch_size=32,
                              num_workers=1,
                              )
        train_minibatches_iterator = iter(val_loader)

        for step in tqdm.tqdm(range(len(val_loader))):

            total_group_members = np.sum([batch.shape[0] for batch in LOGITS_EDIT_TEST[f'y{Y_CURR}'][f'a{A_CURR}']])
            if total_group_members > per_group_repr:
                break
            i, x, y, a = next(train_minibatches_iterator)

            A_MASK = (a == A_CURR)
            Y_MASK = (y == Y_CURR)
            MASK = A_MASK & Y_MASK
            x = x[MASK]
            input_ids = x[:,:,0].cuda()
            one_zero_attention_mask = x[:,:,1].cuda()
            token_type_ids = x[:,:,2].cuda()

            with torch.no_grad():
                logits = bert_updated(input_ids, one_zero_attention_mask=one_zero_attention_mask, token_type_ids=token_type_ids)
                LOGITS_EDIT_TEST[f'y{Y_CURR}'][f'a{A_CURR}'].append(logits[:,0,:].detach().cpu())
            del logits
            torch.cuda.empty_cache()

        LOGITS_EDIT_TEST[f'y{Y_CURR}'][f'a{A_CURR}'] = torch.cat(LOGITS_EDIT_TEST[f'y{Y_CURR}'][f'a{A_CURR}'])[:per_group_repr]

## Confusion Matrix

In [ ]:
# Initialize confusion matrices
conf_matrix_a0 = np.zeros((NUM_CLASSES, NUM_CLASSES), dtype=int)
conf_matrix_a1 = np.zeros((NUM_CLASSES, NUM_CLASSES), dtype=int)

conf_matrix = {ai: np.zeros((NUM_CLASSES, NUM_CLASSES), dtype=int) for ai in a_s}

# Compute confusion matrix for a0 and a1
# for ai in a_s:
conf_matrix_dict = {}

for ai in a_s:
    for true_label in range(NUM_CLASSES):
        logits_ai = LOGITS_EDIT_TEST[y_s[true_label]][ai]
        pred_labels_ai = logits_ai.argmax(dim=1).numpy()
        for pred_label in pred_labels_ai:
            conf_matrix[ai][true_label, pred_label] += 1
    conf_matrix_dict[ai] = pd.DataFrame(conf_matrix[ai], columns=[f'Pred y{i}' for i in range(NUM_CLASSES)], index=[f'True y{i}' for i in range(NUM_CLASSES)]).T / per_group_repr * 100

accuracy_df = pd.concat(conf_matrix_dict, axis=1).round(2)

### For Test ###
accuracy_df_PATH = path_to_root + f'/results/ModelEdits/Final/ConfusionMatrixTest_{DATASET}_seed{SEED}'
accuracy_df.to_csv(accuracy_df_PATH)

# Save W_edit

In [ ]:
W_edit_PATH = path_to_root + f'/results/ModelEdits/Final/W_edit_{DATASET}_seed{SEED}.pth'
torch.save(W_edit , W_edit_PATH)